In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Imports and Utility Functions

In [ ]:
TRAINING_DATA = "/kaggle/input/nk-iv-prediction/train_data.parquet"
TESTING_DATA = "/kaggle/input/nk-iv-prediction/test_data.parquet"
SAMPLE_SUBMISSION_DATA = "/kaggle/input/nk-iv-prediction/sample_submission.csv"

In [ ]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
from tqdm.auto import tqdm

# ─────────────────────────────────────────────────────────────────
# 1.1  Raw SVI total‐variance function and objective
# ─────────────────────────────────────────────────────────────────

def svi_total_variance(params, k_vals):
    """
    Raw SVI total variance:
        w(k) = a + b * [ rho*(k - m) + sqrt((k - m)^2 + sigma^2 ) ].
    params = [a, b, rho, m, sigma]
    k_vals = array of log-moneyness
    Returns array w(k_vals)
    """
    a, b, rho, m, sig = params
    return a + b * (rho * (k_vals - m) + np.sqrt((k_vals - m)**2 + sig**2))


def svi_obj_on_iv(params, k_obs, iv_obs):
    """
    Objective (sum of squared errors on IV) for a single row:
       minimize Σ_i [ iv_obs_i - sqrt( w(k_i) ) ]^2,
    where w(k) = svi_total_variance(params, k).
    """
    w = svi_total_variance(params, k_obs)
    # enforce positive total variance
    if np.any(w <= 0):
        return np.inf
    iv_model = np.sqrt(w)
    return np.sum((iv_model - iv_obs)**2)


# ─────────────────────────────────────────────────────────────────
# 1.2  Per‐row SVI calibration (calls OR puts)
# ─────────────────────────────────────────────────────────────────

def calibrate_svi_for_row(strikes, ivs, underlying):
    """
    Fit SVI for a *single row*'s subset of strikes & IVs.
    Inputs:
      - strikes:  array of strike prices where IV is known
      - ivs:      array of corresponding observed IVs
      - underlying: scalar underlying price for that row
    Returns:
      params (a,b,rho,m,sigma) if fit succeeds; else None
    """
    # If too few points, bail out
    if len(strikes) < 5:
        return None

    # Compute log-moneyness k_i = ln(K / underlying)
    k_obs = np.log(strikes / underlying)
    iv_obs = ivs.astype(float)

    # Initial guess for [a,b,rho,m,sigma]
    # - a0: roughly minimum total variance = (min iv)^2
    v_min = np.min(iv_obs**2)
    a0 = max(v_min * 0.8, 1e-6)  # a small positive floor
    b0 = 0.1                     # small positive slope
    rho0 = 0.0
    m0 = 0.0                     # assume roughly ATM center
    sig0 = 0.1
    x0 = np.array([a0, b0, rho0, m0, sig0], dtype=float)

    # Bounds: a>0, b>0, |rho|<0.999, m in [min(k)-0.5, max(k)+0.5], sigma>0
    bounds = [
        (1e-8, np.inf),                    # a
        (1e-8, np.inf),                    # b
        (-0.999, 0.999),                   # rho
        (np.min(k_obs)-0.5, np.max(k_obs)+0.5),  # m
        (1e-8, 5.0)                        # sigma
    ]

    try:
        res = opt.minimize(
            fun=lambda x: svi_obj_on_iv(x, k_obs, iv_obs),
            x0=x0,
            bounds=bounds,
            method="L-BFGS-B",
            options={"maxiter":200, "ftol":1e-8}
        )
        if (not res.success) or np.any(res.x < 0):
            return None
        return res.x  # fitted [a,b,rho,m,sigma]
    except:
        return None


def fill_svi_smile(params, target_strikes, underlying):
    """
    Given fitted SVI params = [a,b,rho,m,sigma], return IVs at target_strikes:
      iv_pred = sqrt( w(k) )  where  k = ln(K/underlying).
    If params is None, returns NaNs.
    """
    if params is None:
        return np.full(len(target_strikes), np.nan)
    k_all = np.log(target_strikes / underlying)
    w_all = svi_total_variance(params, k_all)
    # ensure no negative variances
    w_all = np.maximum(w_all, 0.0)
    return np.sqrt(w_all)


## 2. Load Test Data & Sample Submission

In [ ]:
# 2.1  Which columns correspond to calls and puts?
#     We will need to parse the column names like "call_iv_26000" → strike=26000.

test_df = pd.read_parquet(TESTING_DATA)
sample_sub = pd.read_csv(SAMPLE_SUBMISSION_DATA)

# Extract the full list of IV‐columns (they appear in sample_submission in order).
iv_columns = [c for c in sample_sub.columns if ("call_iv_" in c) or ("put_iv_" in c)]

# For convenience, build separate sorted lists of all possible strikes:
call_cols = [c for c in iv_columns if c.startswith("call_iv_")]
put_cols  = [c for c in iv_columns if c.startswith("put_iv_")]

# Extract strike values as integers:
call_strikes = sorted(int(c.replace("call_iv_","")) for c in call_cols)
put_strikes  = sorted(int(c.replace("put_iv_","")) for c in put_cols)

# Map column names → strike (dict):
call_col_to_strike = {f"call_iv_{k}": k for k in call_strikes}
put_col_to_strike  = {f"put_iv_{k}":  k for k in put_strikes}

print("Total test rows:", len(test_df))
print("Number of call strikes:", len(call_strikes), "→", call_strikes[:5], "…", call_strikes[-5:])
print("Number of put strikes: ", len(put_strikes),  "→", put_strikes[:5],  "…", put_strikes[-5:])
print(sample_sub.columns.tolist())

## 3. Row-wise Imputation Loop (calls + puts)

In [ ]:
# Prepare an output array of shape (n_rows, n_iv_columns)
n_rows = len(test_df)
n_ivcols = len(iv_columns)

# We'll build a DataFrame exactly matching sample_sub’s shape/order:
out = pd.DataFrame(index=np.arange(n_rows), columns=sample_sub.columns, dtype=float)
out["timestamp"] = sample_sub["timestamp"].values

# To show progress:
pbar = tqdm(total=n_rows, desc="Imputing rows")

for i, row in test_df.iterrows():
    underlying = float(row["underlying"])  # NIFTY50 spot price
    
    # ────────────────────────────────────────────
    # 3.1  Calls: gather observed call IVs for this row
    # ────────────────────────────────────────────
    observed_call_strikes = []
    observed_call_ivs     = []
    missing_call_strikes  = []
    
    for colname, strike in call_col_to_strike.items():
        val = row[colname]
        if np.isfinite(val):
            observed_call_strikes.append(strike)
            observed_call_ivs.append(val)
        else:
            missing_call_strikes.append(strike)
    
    observed_call_strikes = np.array(observed_call_strikes, dtype=float)
    observed_call_ivs     = np.array(observed_call_ivs, dtype=float)
    missing_call_strikes  = np.array(missing_call_strikes, dtype=float)
    
    # Fit SVI on calls
    params_call = calibrate_svi_for_row(observed_call_strikes, observed_call_ivs, underlying)
    
    # Predict missing calls
    if params_call is not None:
        call_preds = fill_svi_smile(params_call, missing_call_strikes, underlying)
    else:
        # Fallback: simple linear interpolation in k‐space
        if len(observed_call_strikes) >= 2:
            k_obs = np.log(observed_call_strikes / underlying)
            iv_obs = observed_call_ivs
            k_miss = np.log(missing_call_strikes / underlying)
            # np.interp on k_miss (left/right are filled with nearest)
            iv_pred = np.interp(k_miss, k_obs, iv_obs)
            call_preds = iv_pred
        else:
            # Too few points → fill with a constant (e.g. nearest neighbor)
            call_preds = np.full(len(missing_call_strikes), observed_call_ivs.mean() if len(observed_call_ivs)>0 else 0.2)
    
    # Write back into output DataFrame
    #   - For observed indices: keep the original value
    for colname, strike in call_col_to_strike.items():
        if not np.isfinite(row[colname]):
            # find index in missing_call_strikes
            idx = np.where(missing_call_strikes == strike)[0][0]
            out.at[i, colname] = call_preds[idx]
        else:
            out.at[i, colname] = row[colname]
    
    # ────────────────────────────────────────────
    # 3.2  Puts: gather observed put IVs for this row
    # ────────────────────────────────────────────
    observed_put_strikes = []
    observed_put_ivs     = []
    missing_put_strikes  = []
    
    for colname, strike in put_col_to_strike.items():
        val = row[colname]
        if np.isfinite(val):
            observed_put_strikes.append(strike)
            observed_put_ivs.append(val)
        else:
            missing_put_strikes.append(strike)
    
    observed_put_strikes = np.array(observed_put_strikes, dtype=float)
    observed_put_ivs     = np.array(observed_put_ivs, dtype=float)
    missing_put_strikes  = np.array(missing_put_strikes, dtype=float)
    
    # Fit SVI on puts
    params_put = calibrate_svi_for_row(observed_put_strikes, observed_put_ivs, underlying)
    
    # Predict missing puts
    if params_put is not None:
        put_preds = fill_svi_smile(params_put, missing_put_strikes, underlying)
    else:
        # Fallback: linear interpolation in k‐space
        if len(observed_put_strikes) >= 2:
            k_obs = np.log(observed_put_strikes / underlying)
            iv_obs = observed_put_ivs
            k_miss = np.log(missing_put_strikes / underlying)
            iv_pred = np.interp(k_miss, k_obs, iv_obs)
            put_preds = iv_pred
        else:
            put_preds = np.full(len(missing_put_strikes), observed_put_ivs.mean() if len(observed_put_ivs)>0 else 0.2)
    
    # Write back into output DataFrame
    for colname, strike in put_col_to_strike.items():
        if not np.isfinite(row[colname]):
            idx = np.where(missing_put_strikes == strike)[0][0]
            out.at[i, colname] = put_preds[idx]
        else:
            out.at[i, colname] = row[colname]
    
    pbar.update(1)
    print("imputed row ",i)

pbar.close()

## 4. Sanity‐check & Export Submission 

In [ ]:
# 4.1  Quick sanity: check no NaNs remain
assert out.isna().sum().sum() == 0, "There are still NaNs in the output!"

# 4.2  Make sure the column order matches sample_submission
out = out[sample_sub.columns]

# 4.3  Write submission.csv
out.to_csv("submission.csv", index=False)
print("Wrote submission.csv with shape", out.shape)

In [24]:
import numpy as np
import pandas as pd

# 6.1  Read the sample submission (with timestamp + IV columns) and our submission
sample_sub = pd.read_csv(SAMPLE_SUBMISSION_DATA)
sub        = pd.read_csv("submission.csv")

# 6.2  Check that shape and column order match exactly
assert sub.shape == sample_sub.shape, f"Shape mismatch: {sub.shape} vs {sample_sub.shape}"
assert list(sub.columns) == list(sample_sub.columns), "Column order or names mismatch!"

# 6.3  Check for any NaNs in our submission
assert sub.isna().sum().sum() == 0, "Submission has NaNs!"

# 6.4  Extract the list of IV columns (everything except 'timestamp')
iv_columns = [c for c in sub.columns if c != "timestamp"]

# 6.5  Confirm that for every IV column, any originally‐observed IV in test_df was preserved exactly
#      → We need test_df loaded in memory (with the same ordering as sub)
test_df = pd.read_parquet(TESTING_DATA)

# Mask of “observed” IV entries in test_df
mask_obs = test_df[iv_columns].notna()

# Compare only those entries:
pred_vals = sub[iv_columns].values
true_vals = test_df[iv_columns].values

diff = pred_vals[mask_obs.values] - true_vals[mask_obs.values]
assert np.allclose(diff, 0.0, atol=1e-12), "Some observed IVs were changed!"

# 6.6  Compute and print RMSE over **only** the observed cells:
rmse_observed = np.sqrt(np.mean((diff) ** 2))
print(f"Observed‐cell RMSE (should be 0): {rmse_observed:.12f}")

# 6.7  Compute and print RMSE over the **filled** (formerly‐masked) cells:
mask_filled = ~mask_obs.values  # True where test_df was NaN
filled_diff = pred_vals[mask_filled]  # no ground truth for these, so we skip
# We cannot compute “true” for masked cells (they’re NaN), so we skip RMSE here.
# If you have a hold‐out on train, compute that separately.

# 6.8  Spot‐check that all IVs are within a reasonable range [0, 5]:
iv_data = sub[iv_columns].values.flatten()
assert (iv_data >= 0.0).all() and (iv_data <= 5.0).all(), "Some IVs are out of a reasonable range!"

print("✔ Sanity check passed: shape, columns, no NaNs, observed IVs untouched, IV range OK.")

Observed‐cell RMSE (should be 0): 0.000000000000
✔ Sanity check passed: shape, columns, no NaNs, observed IVs untouched, IV range OK.


## Flattened long-form transformation

In [26]:
import pandas as pd

# 1. Read the original wide-format train.parquet
train = pd.read_parquet(TRAINING_DATA)

# 2. Identify all IV columns for calls and for puts
iv_cols = [c for c in train.columns if c.startswith("call_iv_") or c.startswith("put_iv_")]
call_cols = [c for c in iv_cols if c.startswith("call_iv_")]
put_cols  = [c for c in iv_cols if c.startswith("put_iv_")]

# 3. Melt calls into long form:
#    Each row: timestamp, underlying, expiry, X0–X41, var="call_iv_K", iv=value, cp_flag=0, strike=K
calls_long = train.melt(
    id_vars=["timestamp", "underlying", "expiry"] + [f"X{i}" for i in range(42)],
    value_vars=call_cols,
    var_name="var", value_name="iv"
)
calls_long["cp_flag"] = 0  # calls
calls_long["strike"] = calls_long["var"].str.replace("call_iv_", "").astype(int)
calls_long = calls_long.drop(columns="var")

# 4. Melt puts into long form similarly:
puts_long = train.melt(
    id_vars=["timestamp", "underlying", "expiry"] + [f"X{i}" for i in range(42)],
    value_vars=put_cols,
    var_name="var", value_name="iv"
)
puts_long["cp_flag"] = 1  # puts
puts_long["strike"] = puts_long["var"].str.replace("put_iv_", "").astype(int)
puts_long = puts_long.drop(columns="var")

# 5. Concatenate calls_long and puts_long into a single DataFrame
train_long = pd.concat([calls_long, puts_long], ignore_index=True)

# 6. Reset index and inspect
train_long = train_long.reset_index(drop=True)
print("Long‐form train shape:", train_long.shape)
print(train_long.head())


Long‐form train shape: (9273680, 48)
             timestamp  underlying      expiry        X0        X1  \
0  1745296089000000000     24160.9  2025-04-24 -0.092103 -0.084458   
1  1745304077000000000     24188.1  2025-04-24 -0.013699 -0.023263   
2  1745313495000000000     24148.6  2025-04-24 -0.395427 -0.056440   
3  1745313499000000000     24147.4  2025-04-24  0.007829 -0.086614   
4  1745313608000000000     24155.9  2025-04-24  0.012404 -0.005619   

             X2        X3            X4        X5        X6  ...       X35  \
0  1.025842e+05  0.001655 -1.379624e+06  0.027959 -0.020240  ...  0.024715   
1  1.086423e+05 -0.004734  2.528508e+06 -0.006439 -0.011416  ... -0.004020   
2 -1.194717e+06  0.005011 -1.185146e+08 -0.027625 -0.002189  ... -0.035342   
3  5.936540e+05  0.001347  3.826919e+06 -0.052881 -0.015226  ... -0.045472   
4  2.419948e+05 -0.001172  4.596446e+06  0.039326  0.015934  ...  0.044814   

        X36       X37           X38           X39            X40  \
0  0.